# BarrelNet Inference Notebook
This Notebook will run the code to run an inference pipeline for the trained Model, given a single Barrel point cloud sample as input. 

In [1]:
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from tqdm import tqdm
import os
import torch
import torch.optim as optim
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import uuid
import roma
from data import generate_cylinder_pts, prepare_point_cloud, normalize_pc, CylinderData
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import Dataset, DataLoader
from pointnet_utils import PointNetEncoder, feature_transform_reguliarzer
from barrelnet import BarrelNet

In [6]:
## Load Model 
model_path = 'weights/pointnet.pth'

pointnet = BarrelNet(k=4, normal_channel=False).cuda()
pointnet.load_state_dict(torch.load(model_path))
pointnet.eval()

## TODO Write code here to prepare a random input point cloud for inference. 
pts = torch.rand(1100).cuda() ## Replace this with dust3r point cloud extracted for the barrel
pts, scale = normalize_pc(pts[0])
with torch.no_grad():
    radius_pred, axis_pred = pointnet(pts)

In [7]:
axis_pred

tensor([[-0.5774,  0.5774,  0.5774]], device='cuda:0')